# 2. Intro to Extended Kalman Filter Project

## 2.1 - Project Introduction

Now that you have learned how the extended Kalman filter works, you are going to implement the extended Kalman filter in C++. We are providing `simulated lidar and radar measurements detecting a bicycle that travels around your vehicle`. You will use a Kalman filter, lidar measurements and radar measurements to track the bicycle's position and velocity.

The first step is to `download the simulator`, which contains all the projects for Self-Driving Car Nanodegree. More detailed instruction about setting up the simulator with `uWebSocketIO` can be found at the end of this section.

`Lidar` measurements are `red circles`,` radar` measurements are `blue circles` with an arrow pointing in the direction of the observed angle, and `estimation markers` are `green triangles`. 

The video below shows what the simulator looks like when a c++ script is using its Kalman filter to track the object. The simulator provides the script the measured data (either lidar or radar), and the script feeds back the measured estimation marker, and RMSE values from its Kalman filter.

In [9]:
from IPython.display import HTML
HTML("""
<video width="600" height="400" controls>
  <source src='_Videos/2 - T2 P1 EKF.mp4'>
</video>
""")

## 2.2 - Example of Tracking with Lidar

In [4]:
HTML("""
<video width="640" height="360" controls>
  <source src='_Videos/3_Data_collected.mp4'>
</video>
""")

# 3. Data File for EKF project

## 3.1 - Explanation of the Data File

The github repo contains one data file:

> obj_pose-laser-radar-synthetic-input.txt

Here is a screenshot of the first data file:

![](_Images/3_1_screenshot.png)
Screenshot of Data File

The simulator will be using this data file, and feed main.cpp values from it one line at a time.

Each row represents a sensor measurement where the first column tells you if the measurement comes from radar (R) or lidar (L).

For a row containing radar data, the columns are: sensor_type, rho_measured, phi_measured, rhodot_measured, timestamp, x_groundtruth, y_groundtruth, vx_groundtruth, vy_groundtruth, yaw_groundtruth, yawrate_groundtruth.

For a row containing lidar data, the columns are: sensor_type, x_measured, y_measured, timestamp, x_groundtruth, y_groundtruth, vx_groundtruth, vy_groundtruth, yaw_groundtruth, yawrate_groundtruth.

Whereas radar has three measurements (rho, phi, rhodot), lidar has two measurements (x, y).

You will use the measurement values and timestamp in your Kalman filter algorithm. Groundtruth, which represents the actual path the bicycle took, is for calculating root mean squared error.

You do not need to worry about `yaw` and `yaw rate` ground truth values.

# 3.2 - Reading in the Data

We have provided code that will read in and parse the data files for you. This code is in the main.cpp file. The main.cpp file creates instances of a MeasurementPackage.

If you look inside 'main.cpp', you will see code like:

```C++
MeasurementPackage meas_package;
meas_package.sensor_type_ = MeasurementPackage::LASER;
meas_package.raw_measurements_ = VectorXd(2);
meas_package.raw_measurements_ << px, py;
meas_package.timestamp_ = timestamp;
```

and

```C++
vector<VectorXd> ground_truth;
VectorXd gt_values(4);
gt_values(0) = x_gt;
gt_values(1) = y_gt; 
gt_values(2) = vx_gt;
gt_values(3) = vy_gt;
ground_truth.push_back(gt_values);
```

The code reads in the data file line by line. The measurement data for each line gets pushed onto a measurement_pack_list. The ground truth $[p_x, p_y, v_x, v_y]$ for each line in the data file gets pushed ontoground_truthso RMSE can be calculated later from tools.cpp.

# 4. File Structure

# 4.1 - Overview of a Kalman Filter: Initialize, Predict, Update

To review what we learned in the extended Kalman filter lectures, let's discuss the three main steps for programming a Kalman filter:

- `initializing` Kalman filter variables
- `predicting` where our object is going to be after a time step $\Delta{t}$
- `updating` where our object is based on sensor measurements

Then the prediction and update steps repeat themselves in a loop.

To measure how well our Kalman filter performs, we will then calculate root mean squared error comparing the Kalman filter results with the provided ground truth.

These `three steps (initialize, predict, update)` plus calculating `RMSE` encapsulate the entire extended Kalman filter project.

# 4.2 - Files in the Github src Folder

The files you need to work with are in the src folder of the github repository.

- `main.cpp` - communicates with the `Term 2 Simulator` receiving data measurements, calls a function to run the Kalman filter, calls a function to calculate RMSE
- `FusionEKF.cpp` - initializes the filter, calls the predict function, calls the update function
- `kalman_filter.cpp`- defines the `predict function`, the `update function for lidar`, and t`he update function for radar`
- `tools.cpp`v- function to calculate RMSE and the Jacobian matrix

The only files you need to modify are `FusionEKF.cpp`, `kalman_filter.cpp`, and `tools.cpp`.

# 4.3 - How the Files Relate to Each Other

Here is a brief overview of what happens when you run the code files:

- `Main.cpp` reads in the data and sends a `sensor measurement` to `FusionEKF.cpp`
- `FusionEKF.cpp` takes the sensor data and initializes variables and updates variables. The Kalman filter equations are not in this file. FusionEKF.cpp has a variable called ekf_, which is an instance of a KalmanFilter class. The ekf_ will hold the matrix and vector values. You will also use the ekf_ instance to call the predict and update equations.
- `The KalmanFilter` class is defined in kalman_filter.cpp and kalman_filter.h. You will only need to modify 'kalman_filter.cpp', which contains functions for the prediction and update steps.

# 5. Main.cpp

# 6. Project Code

Compiling and Running Your Code

Take a look at the github repo [README file](https://github.com/udacity/CarND-Extended-Kalman-Filter-Project/blob/master/README.md) for instructions on how to compile and run your code.

# 7. Tips and Tricks

# 8. Project REsources

# 9. Project Instruction for workspace

# **10. Project Instructions for local setup

# ** 12. uWebSocketIO Starter guide

## **15. Environment Setup (Mac)

# **16. Compiling and Running the Project